In [1]:
using POMDPs
using Random # for AbstractRNG
using POMDPModelTools
using Pkg

using DataFrames
Pkg.add("JSON")

POMDPs.add_registry()
using Pkg; Pkg.add("DiscreteValueIteration")

using DiscreteValueIteration

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
 Installed WebSockets ───────── v1.3.1
 Installed TranscodingStreams ─ v0.9.0
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [3bb67fe8] ↑ TranscodingStreams v0.8.1 ⇒ v0.9.0
  [104b5d7c] ↑ WebSockets v1.3.0 ⇒ v1.3.1
   Cloning registry from "https://github.com/JuliaPOMDP/Registry"
Fetching: [========================================>]  100.0 %.0 %=========>                               ]  21.4 %]  42.7 %==========================>              ]  64.1 %]  85.3 %

┌ Info: registry `JuliaPOMDP` already exists in `~/.julia/registries`
└ @ POMDPs /Users/efan/.julia/packages/POMDPs/JiYXY/src/utils.jl:65


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [2]:
using Pkg
Pkg.add("StaticArrays")
using POMDPSimulators
using POMDPPolicies

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [ ]:

include("./ParallelChains.jl")

In [ ]:
num_chains = 5
num_states = 6
r_mean = 5.0
r_std = 2.0

true_values = DataFrame(theta = [r_mean for i in 1:num_chains],
                        std = [r_std  for i in 1:num_chains])

In [37]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);

PParallelChainMDP() = PParallelChainMDP(num_states+1,num_chains, .9,

        true_values.theta,
        true_values.std,
        MersenneTwister(1235))
m = PParallelChainMDP()

# policy that maps every input to an action
policy = FunctionPolicy(s->3)
for i in 1:3
    for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
      render(m, (s,a,r))
      println("s,a,r:($s,$a,$r)")
    end
end

s,a,r:([1, 1],3,0)
s,a,r:([3, 2],3,0)
s,a,r:([3, 3],3,0)
s,a,r:([3, 4],3,0)
s,a,r:([3, 5],3,0)
s,a,r:([3, 6],3,0)
s,a,r:([3, 7],3,0)
s,a,r:([3, 8],3,0)
s,a,r:([3, 9],3,0)
s,a,r:([3, 10],3,0)
s,a,r:([1, 1],3,0)
s,a,r:([3, 2],3,0)
s,a,r:([3, 3],3,0)
s,a,r:([3, 4],3,0)
s,a,r:([3, 5],3,0)
s,a,r:([3, 6],3,0)
s,a,r:([3, 7],3,0)
s,a,r:([3, 8],3,0)
s,a,r:([3, 9],3,0)
s,a,r:([3, 10],3,0)
s,a,r:([1, 1],3,0)
s,a,r:([3, 2],3,0)
s,a,r:([3, 3],3,0)
s,a,r:([3, 4],3,0)
s,a,r:([3, 5],3,0)
s,a,r:([3, 6],3,0)
s,a,r:([3, 7],3,0)
s,a,r:([3, 8],3,0)
s,a,r:([3, 9],3,0)
s,a,r:([3, 10],3,0)


In [ ]:
curry(f, a) = (xs...) -> f(a, xs...)

In [ ]:
using POMDPSimulators
using POMDPPolicies
rng = MersenneTwister(1235);

num_chains = 10
num_states = 30
r_mean = 5.0
r_std = 2.0
@requirements_info ValueIterationSolver() PParallelChainMDP(num_states+1,num_chains, .9,

        randn(rng, Float32, num_chains) .* r_std .+ r_mean)

In [51]:
num_chains=2
r_mean = 0
r_std = 1
## sigma is 1
## Reward is noisy ~ N(theta_c, sigma^2)
## Initial prior mean is N(theta_c, sigma^2 + c)
rng = MersenneTwister(1235)
th_c = [randn(rng, Float64, 1)[1] * (100 + i) for i in 1:num_chains]
println(th_c)
true_values = DataFrame(theta = th_c,
                         std = [r_std  for i in 1:num_chains])
mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        true_values.theta,
        true_values.std, 
        rng)
# @requirements_info ValueIterationSolver() mdp
solver = ValueIterationSolver(max_iterations=100, belres=1e-6, include_Q=true, verbose=true) # initializes the Solver type
vip = solve(solver, mdp) # runs value iterations


[-11.0276, 18.6375]
reward: -12.818057603057788
reward: 19.165515612310358
[Iteration 1   ] residual:          0 | iteration runtime:      0.307 ms, (  0.000307 s total)


ValueIterationPolicy{Float64}([0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 2], true, PParallelChainMDP(31, 2, 0.9, [-11.0276, 18.6375], [1.0, 1.0], MersenneTwister(UInt32[0x000004d3], Random.DSFMT.DSFMT_state(Int32[1530564531, 1073184989, -470697644, 1073462633, -1352142760, 1073554780, 1647934450, 1073384872, 1869502942, 1072864888  …  1558688637, 1073219162, 934987498, 1073204542, 505616865, -1680158283, 679873689, -1200220289, 382, 0]), [1.09516, 1.2709, 1.90632, 1.83358, 1.94505, 1.44367, 1.90458, 1.9416, 1.03759, 1.99608  …  1.97544, 1.35875, 1.80559, 1.15357, 1.28736, 1.44507, 1.12787, 1.11129, 1.50155, 1.48761], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [50]:
vip.qmat

62×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 ⋮       
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [ ]:
LinearIndices((num_chains, num_states))[1,1]

In [ ]:

for i in 1:num_chains
    println("q value for (1,1),$i is $(vip.qmat[LinearIndices((num_chains, num_states))[1,1],i])")
    println("q value for $((i,num_states)),1 is $(vip.qmat[LinearIndices((num_chains, num_states))[i,num_states],1])")
end

In [ ]:
mdp.Rs

In [36]:
using POMDPSimulators
using POMDPPolicies



for (s, a, r) in stepthrough(mdp, vip, "s,a,r", max_steps=40)
    render(mdp, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end



s,a,r:([1, 1],1,0)
s,a,r:([1, 2],1,0)
s,a,r:([1, 3],1,0)
s,a,r:([1, 4],1,0)
s,a,r:([1, 5],1,0)
s,a,r:([1, 6],1,0)
s,a,r:([1, 7],1,0)
s,a,r:([1, 8],1,0)
s,a,r:([1, 9],1,0)
s,a,r:([1, 10],1,0)
s,a,r:([1, 11],1,0)
s,a,r:([1, 12],1,0)
s,a,r:([1, 13],1,0)
s,a,r:([1, 14],1,0)
s,a,r:([1, 15],1,0)
s,a,r:([1, 16],1,0)
s,a,r:([1, 17],1,0)
s,a,r:([1, 18],1,0)
s,a,r:([1, 19],1,0)
s,a,r:([1, 20],1,0)
s,a,r:([1, 21],1,0)
s,a,r:([1, 22],1,0)
s,a,r:([1, 23],1,0)
s,a,r:([1, 24],1,0)
s,a,r:([1, 25],1,0)
s,a,r:([1, 26],1,0)
s,a,r:([1, 27],1,0)
s,a,r:([1, 28],1,0)
s,a,r:([1, 29],1,0)
reward: [-11.1731]
s,a,r:([1, 30],1,0)


In [53]:
function ucb_pol(li, policy, priors, i, actions, s)
    best_action = action(policy, s)
    # println("in state: $s, best action:$best_action")
    return action(policy, s)
end

ucb_pol (generic function with 1 method)

In [60]:
# li = LinearIndices((num_chains, num_states))
function ucb_update_priors(priors, history, i)
    return priors
end

ucb_update_priors (generic function with 1 method)

In [55]:
curry(f, x) = (xs...) -> f(x, xs...)
function ucb_mdp_builder(true_mdp, priors, i, num_states, num_chains, steps)
  # build mdp
  new_means = priors.theta + priors.std
  println("new means: $new_means, std: $(priors.std), num_chains:$num_chains, num_states:$num_states, i:$i, mdp:$(typeof(true_mdp))")

  mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        new_means,
        priors.std,
        MersenneTwister(1235))
  println("#2")
  # solve mdp
  println("#3")
  solver = ValueIterationSolver(max_iterations=1000, belres=1e-6, include_Q=true) # initializes the Solver type
  vip = solve(solver, mdp)
  li = LinearIndices((num_chains, num_states))
  #print("policy q: $(vip.qmat)")
  new_policy = FunctionPolicy(curry(curry(curry(curry(curry(ucb_pol, li), vip), priors),i), actions))
  # print("new policy: $new_policy")
  # return iterator solved mdp
  return Iterators.Stateful(stepthrough(deepcopy(true_mdp), new_policy, "s,a,r,sp,t",
                            max_steps=steps))
end

ucb_mdp_builder (generic function with 1 method)

In [29]:

do_update_priors() =  false
update_priors() = false

update_priors (generic function with 1 method)

In [59]:


# UCB
using Distributions

include("./ParallelChains.jl")
# setup constants here
num_agents = 3
num_chains = 10
num_states = 30
epochs = 1
H = 30

# So best idea:
# for ucb:
# we generate an mdp with mu + sigma rewards (priors) for each end node.
#  we collect history until the criteria in PAC-EXPLORE then update the prior
# 
# for thompson sampling
# at each time step (might skip this for this problem and save for max rew path) 
#   take rewards and use to create a posterior.
# for seed sampling
# at beginning of episode
#   each agent generates a new random "seed" 
#   at each time step
#     each agent generates a new mdp based on a deterministic mapping from seed to rewards (which also takes
#              history into account)
# this needs to update the actual reward.

## PC notes
## theta_c ~ N(0, 100 + c)
r_mean = 0
r_std = 1
## sigma is 1
## Reward is noisy ~ N(theta_c, sigma^2)
## Initial prior mean is N(theta_c, sigma^2 + c)
rng = MersenneTwister(1235)
th_c = [randn(rng, Float64, 1)[1] * (100 + i) for i in 1:num_chains]
println(th_c)
true_values = DataFrame(theta = th_c,
                         std = [r_std  for i in 1:num_chains])
print("true mdp: $true_values")
priors = DataFrame(theta = [true_values.theta[i] for i in 1:num_chains],
                    std = [r_std + i for i in 1:num_chains])
mdp = PParallelChainMDP(num_states+1,num_chains, .9,
        true_values.theta,
        true_values.std, 
        rng)
do_update_priors() =  false
# priors = randn(rng, Float32, num_chains) .* (r_std + c) .+ r_mean
hist = run_chain!(
           mdp_iter_builder=ucb_mdp_builder,
           true_mdp=mdp,
           do_update_priors=do_update_priors,
           update_priors=update_priors,
           priors=priors,
           n_agents=num_agents,
           num_states=num_states,
           num_chains=num_chains,
           epochs=epochs,
           steps=H) #,
           #rev_action_map=rev_action_map)
#print(Q_tables)
println("(e,i,t,st,r)")
hist

[-11.0276, 18.6375, -184.418, 54.9143, -227.549, 82.2941, -10.8835, 19.4219, 58.0705, -21.0394]
true mdp: 10×2 DataFrame
│ Row │ theta    │ std   │
│     │ Float64  │ Int64 │
├─────┼──────────┼───────┤
│ 1   │ -11.0276 │ 1     │
│ 2   │ 18.6375  │ 1     │
│ 3   │ -184.418 │ 1     │
│ 4   │ 54.9143  │ 1     │
│ 5   │ -227.549 │ 1     │
│ 6   │ 82.2941  │ 1     │
│ 7   │ -10.8835 │ 1     │
│ 8   │ 19.4219  │ 1     │
│ 9   │ 58.0705  │ 1     │
│ 10  │ -21.0394 │ 1     │new means: [-9.0276, 21.6375, -180.418, 59.9143, -221.549, 89.2941, -2.88354, 28.4219, 68.0705, -10.0394], std: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], num_chains:10, num_states:30, i:1, mdp:PParallelChainMDP
#2
#3
e: 1, t: 1, agent 1, result: (s = [1, 1], a = 6, r = 0, sp = [6, 2], t = 1)
new means: [-9.0276, 21.6375, -180.418, 59.9143, -221.549, 89.2941, -2.88354, 28.4219, 68.0705, -10.0394], std: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], num_chains:10, num_states:30, i:2, mdp:PParallelChainMDP
#2
#3
e: 1, t: 2, agent 1, result: (s = [

90-element Array{Any,1}:
 (1, 1, 1, [1, 1], 0)                  
 (1, 1, 2, [6, 2], 0)                  
 (1, 2, 1, [1, 1], 0)                  
 (1, 1, 3, [6, 3], 0)                  
 (1, 2, 2, [6, 2], 0)                  
 (1, 3, 1, [1, 1], 0)                  
 (1, 1, 4, [6, 4], 0)                  
 (1, 2, 3, [6, 3], 0)                  
 (1, 3, 2, [6, 2], 0)                  
 (1, 1, 5, [6, 5], 0)                  
 (1, 2, 4, [6, 4], 0)                  
 (1, 3, 3, [6, 3], 0)                  
 (1, 1, 6, [6, 6], 0)                  
 ⋮                                     
 (1, 1, 28, [6, 28], 0)                
 (1, 2, 27, [6, 27], 0)                
 (1, 3, 26, [6, 26], 0)                
 (1, 1, 29, [6, 29], 0)                
 (1, 2, 28, [6, 28], 0)                
 (1, 3, 27, [6, 27], 0)                
 (1, 1, 30, [6, 30], 81.30541692118248)
 (1, 2, 29, [6, 29], 0)                
 (1, 3, 28, [6, 28], 0)                
 (1, 2, 30, [6, 30], 81.30541692118248)
 (1, 3, 29, [6,